**извлечение признаков из текста на естественном языке**

токенизация и очистка текста


Евгений Борисов borisov.e@solarl.ru

## библиотеки

In [1]:
import numpy as np
np.set_printoptions(precision=2) # вывод на печать до 2 знака
import pandas as pd
import re
import gzip

In [2]:
pd.options.display.max_colwidth = 200  

## загружаем тексты

In [4]:
# загружаем тексты
df = pd.read_pickle('../data/text/news.pkl.gz')
print('записей:',len(df))

записей: 3196


In [5]:
df.sample(5)

,text,tag
769,Опередить инфаркт. В Витебске появился новый комплекс для лечения болезней сердца\n\n7 декабря 2016 в 8:03\n\nОНТ\n\nНовое оборудование появилось в ангиографическом кабинете областного кардиоцентр...,social
2847,"Телеканал ""Пятница"" подал иск о защите репутации к пользователям социальной сети Facebook, которые обсуждали на страницах соцсети программу ""Ревизорро"", пишут ""Ведомости"". Ведущие ""Ревизорро"" пров...",incident
1283,"Путин пошутил о развитии телепортации в России\n\n9 декабря 2016 в 9:37\n\n42.TUT.BY\n\nПрезидент России Владимир Путин пошутил, предложив поддерживать версию о том, что Москва разрабатывает телеп...",tech
38,"Легендарная фигуристка Ирина Роднина намерена стать главным олимпийцем\nстраны. Она будет одним из семи претендентов на пост президента Олимпийского\nкомитета России на выборах 22 декабря. О том, ...",sport
391,"Residents.ru перестает существовать В начале ноября участники проекта\nResidents.ru объявили о закрытии своего детища, существующего с 2003\nгода. Отныне Kiriloff, Spirit, Berg и Feel расходятся, ...",culture


In [6]:
# df['text'].str.len().describe()
df['tag'].drop_duplicates().count()

13

## токенизация и очистка

In [12]:
# with gzip.open('../data/text/stop-nltk.txt.gz','rt',encoding='utf-8') as f: 
#     stopwords = set([ w.strip() for w in  f.read().split() if w.strip() ] )

# print('количество стоп-слов:',len(stopwords))
# sorted(stopwords)

from Stemmer import Stemmer
# pacman -S python-pystemmer
# pip install pystemmer

In [9]:
# from nltk.stem.snowball import SnowballStemmer
# from nltk.corpus import stopwords as nltk_stopwords

# stopwords = set(nltk_stopwords.words('russian') )

In [13]:
%%time 

df['text_clean'] = df['text'].str.lower() # приведение в lowercase

# замена символов-разделителей (-,_) на пробел
df['text_clean'] = df['text_clean'].apply(lambda s: re.sub( r'\W', ' ', s))
df['text_clean'] = df['text_clean'].apply(lambda s: re.sub( r'_', ' ', s))

# замена цифр
df['text_clean'] = df['text_clean'].apply(lambda s: re.sub( r'\b\d+\b', ' ', s))

# делим строки на слова (токенизация)
df['text_clean'] = df['text_clean'].apply(lambda t: [ w.strip() for w in t.split() if len(w.strip())>2 ] )

# удаление лишних слов 
#df['text_clean'] = df['text_clean'].apply(lambda t:[w for w in t if w not in stopwords])

# стемминг, выделение основы слова
df['text_clean'] = df['text_clean'].apply( lambda t: Stemmer('russian').stemWords(t) )
#df['text_clean'] = df['text_clean'].apply(lambda t:[ SnowballStemmer('russian').stem(w) for w in t if w])


CPU times: user 3.52 s, sys: 65.7 ms, total: 3.59 s
Wall time: 3.61 s


In [14]:
df[['text_clean']].sample(3)

,text_clean
1674,"[экс, президент, ссср, миха, горбач, заяв, что, границ, союз, может, появ, нов, объединен, котор, добровольн, войдут, бывш, советск, республик, отмет, что, качеств, возврат, ссср, так, невозможн, ..."
1475,"[ки, декабр, лидер, радикальн, парт, украин, олег, ляшк, раскритикова, надежд, савченк, встреч, глав, луганск, донецк, народн, республик, игор, плотницк, александр, захарченк, суд, текст, опублико..."
1920,"[президент, сша, барак, обам, арендова, офис, здан, всемирн, фонд, дик, природ, wwf, вашингтон, сообща, газет, washington, post, ссылк, источник, заключен, соглашен, аренд, пок, был, объявл, публи..."
